In [1]:
import math, random
import numpy as np
from tqdm import tqdm

In [5]:
def init(params: dict):
    data, shift_count = np.zeros((params["N"], params["D"]), dtype = int), np.zeros((params["D"], 5))
    shift_count[:,0] = params["N"]
    return data, shift_count


def constraint_error(shift_count: np.array, d: int, s: int, params: dict):
    if s != 0:
        res = 0
        if params["alpha"] <= shift_count[d, s] <= params["beta"]: return res
        elif params["alpha"] > shift_count[d, s]: return params["alpha"] - shift_count[d, s]
        elif params["beta"] < shift_count[d, s]: return params["beta"] - shift_count[d, s]
    elif s == 0:
        min_off = params["N"] - 4 * params["beta"]
        max_off = params["N"] - 4 * params["alpha"]
        if min_off <= shift_count[d, s] <= max_off: return 0
        elif shift_count[d, s] < min_off: return min_off - shift_count[d, s]
        elif max_off >= 0 and shift_count[d, s] > max_off: return max_off - shift_count[d, s]


def check(data:np.array, shift_count: np.array, params: dict):
    for n in range(params["N"]):
        for d in range(params["D"] - 1):
            if data[n][d] == 4 and data[n][d + 1] != 0: return False

    for d in range(params["D"]):
        for s in range(1, 5):
            ce = constraint_error(shift_count, d, s, params)
            if ce > 0: return False
    return True

def objective(data: np.array):
    night_shift = data == 4
    night_shift = np.sum(night_shift, axis = 1)
    return np.max(night_shift)


def f(data: np.array, shift_count: np.array, n: int, d: int, s: int, params: dict):
    night_shifts_score = 0
        
    error_score = constraint_error(shift_count, d, s, params)
    return error_score - night_shifts_score


leaky_relu = lambda x: np.where(x > 0, x, x * 0.01)

EPS = 0.0001
def choose(fs: dict, avail_shifts: list):
    if len(avail_shifts) == 1: return avail_shifts[0]
    probs = np.array([fs[s] for s in avail_shifts])
    probs -= np.min(probs)
    
    #print(probs)
    norm_fact = np.sum(probs)
    if norm_fact == 0: return random.choice(avail_shifts)
    normed_probs = probs / norm_fact
    try:
        return np.random.choice(avail_shifts, 1, p=normed_probs)[0]
    except: print(probs)
    


def optimize(params: dict, F: list, num_reps: int):
    data, shift_count = init(params)
    
    iteration = 0
    saved_n = None
    saved_d = None
    BEST = 1e8
    BEST_SCHEDULE = None
    with tqdm(total = num_reps) as pbar:
        while iteration < num_reps:
            if saved_n is not None and saved_d is not None and saved_d < params["D"] - 1:
                n = saved_n
                d = saved_d + 1
                #print("yo", data[n, d - 1], d - 1)
            else: 
                n = random.choice(range(params["N"]))
                d = random.choice(range(params["D"]))
            
            if d in F[n]: 
                saved_n = None
                saved_d = None
                continue
                
            prev_shift = data[n, d]
            if shift_count[d, prev_shift] > 0: shift_count[d, prev_shift] -= 1
                
            avail_shifts = range(0, 5)
            if d - 1 >= 0 and data[n, d - 1] == 4: 
                #print("okay")
                avail_shifts = [0]
            fs = {s: 0 for s in avail_shifts}
            norm_fact = 0
            for s in avail_shifts:
                fs[s] = f(data, shift_count, n, d, s, params)
                
            chosen_shift = choose(fs, avail_shifts)
            #print(fs, n, d, chosen_shift)
            if chosen_shift == 4:
                saved_n = n
                saved_d = d
            else: 
                saved_n = None
                saved_d = None
                
            data[n, d] = chosen_shift
            shift_count[d, chosen_shift] += 1
            #if iteration % 1000 == 0: print(iteration, fs, chosen_shift)


            if check(data, shift_count, params):
                #print("yeah")
                score = objective(data)
                if score < BEST: 
                    print("found solution, score of solution:", score)
                    BEST = score
                    BEST_SCHEDULE = data.copy()
                    if BEST == 1: 
                        print("found best possible solution, early stopped")
                        break

            iteration += 1
            pbar.update()
            #pbar.set_postfix({"probs": " ".join([str(round(p, 2)) for p in fs.values()])})
        
    return BEST, BEST_SCHEDULE

In [23]:
params = {"N": 50, "D": 5, "alpha": 10, "beta": 10}
"""F = [
    [],
    [3],
    [],
    [],
    [],
    [],
    [],
    [4],
    [],
]"""
F = [[] for index in range(params["N"])]
test = optimize(params, F, 100000)

  4%|███▎                                                                      | 4500/100000 [00:00<00:20, 4677.12it/s]

found solution, score of solution: 2


100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:29<00:00, 3367.81it/s]


In [24]:
test

(2,
 array([[1, 3, 4, 0, 2],
        [3, 4, 0, 2, 4],
        [1, 3, 1, 4, 0],
        [1, 1, 2, 3, 1],
        [2, 2, 4, 0, 4],
        [3, 4, 0, 1, 1],
        [4, 0, 3, 4, 0],
        [2, 2, 3, 2, 3],
        [2, 3, 4, 0, 3],
        [2, 2, 1, 4, 0],
        [4, 0, 4, 0, 1],
        [0, 1, 4, 0, 3],
        [3, 4, 0, 2, 3],
        [4, 0, 3, 2, 1],
        [2, 1, 2, 4, 0],
        [3, 2, 4, 0, 2],
        [1, 3, 3, 4, 0],
        [1, 3, 2, 4, 0],
        [4, 0, 2, 1, 3],
        [3, 3, 1, 1, 3],
        [2, 2, 4, 0, 3],
        [2, 2, 4, 0, 1],
        [0, 3, 3, 1, 2],
        [1, 4, 0, 3, 3],
        [0, 3, 3, 1, 2],
        [4, 0, 3, 4, 0],
        [0, 4, 0, 3, 1],
        [0, 4, 0, 3, 4],
        [0, 3, 1, 1, 2],
        [4, 0, 2, 4, 0],
        [0, 2, 2, 4, 0],
        [0, 2, 1, 2, 1],
        [1, 1, 2, 1, 3],
        [3, 4, 0, 3, 4],
        [4, 0, 2, 3, 4],
        [2, 4, 0, 3, 2],
        [1, 1, 4, 0, 4],
        [0, 1, 3, 1, 1],
        [1, 1, 4, 0, 3],
        [4, 0, 1, 2, 